# Dynamic webscraping of otodom.pl

#### Reference project:
https://github.com/rozek1997/otodom-scrapper/blob/master/src/main.py
#### Selenium after:
https://www.jcchouinard.com/learn-selenium-python-seo-automation/

In [13]:
import os
import pandas as pd
import re
from dataclasses import dataclass
from datetime import date
from fake_useragent import UserAgent
from IPython.core.debugger import set_trace
from math import ceil
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
#from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

In [14]:
hostURL = 'otodom.pl'
#ochota_rentURL = 'https://www.otodom.pl/pl/oferty/wynajem/mieszkanie/warszawa/ochota?ownerTypeSingleSelect=ALL&distanceRadius=0&locations=%5Bdistricts_6-40%5D&viewType=listing'
ochota_rentURL = 'https://www.otodom.pl/pl/wyniki/wynajem/mieszkanie/mazowieckie/warszawa/warszawa/warszawa/ochota?distanceRadius=0&viewType=listing'
ochota_sellURL = 'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/mazowieckie/warszawa/warszawa/warszawa/ochota?ownerTypeSingleSelect=ALL&distanceRadius=0&by=DEFAULT&direction=DESC&viewType=listing'
wlochy_rentURL = 'https://www.otodom.pl/pl/wyniki/wynajem/mieszkanie/mazowieckie/warszawa/warszawa/warszawa/wlochy?ownerTypeSingleSelect=ALL&distanceRadius=0&viewType=listing'
wlochy_sellURL = 'https://www.otodom.pl/pl/wyniki/sprzedaz/mieszkanie/mazowieckie/warszawa/warszawa/warszawa/wlochy?ownerTypeSingleSelect=ALL&distanceRadius=0&viewType=listing'
def pageURL(url, page_number):
    queryURL = ('%s&page=%d' % (url, page_number))
    return queryURL

In [15]:
# To play sound when wanted (f.e. at the end of a function) simply invoke notify
from IPython.display import Audio
import numpy as np

# ---- Examples of other sounds to choose ----
#wave1 = np.sin(1*np.pi*400*np.arange(10000*0.5)/10000)  # 0.5 means will be played for 0.5 seconds
#notify = Audio(wave1, rate=10000, autoplay=True)
#Audio(url='https://wav-sounds.com/wp-content/uploads/2017/09/Email-01.wav', autoplay=True)  -->plays sound from url
#sound_file = './sound/beep.wav'  --> path to a sound
#notify  # --> a way to tell IPython to play sound now, put in in a code whenever sound is to be played

# source: https://gist.github.com/tamsanh/a658c1b29b8cba7d782a8b3aed685a24
framerate = 4410
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(2*np.pi*300*t) + np.sin(2*np.pi*240*t)
notify = Audio(audio_data, rate=framerate, autoplay=True)

In [16]:
##############################################################
#    For testing notify, uncomment below and execute cell
##############################################################

#notify

In [17]:
# To play a sound whenever exception is thrown
# source: https://gist.github.com/krassowski/c6373c155e264bedd7581129a30c7657
from IPython.display import Audio, display

wave2 = np.sin(3*np.pi*400*np.arange(10000*0.5)/10000)

def play_sound(self, etype, value, tb, tb_offset=None):
    self.showtraceback((etype, value, tb), tb_offset=tb_offset)
    display(Audio(wave2, rate=10000, autoplay=True))
    
#get_ipython().set_custom_exc((ZeroDivisionError,), play_sound)  # any Error(s) can be listed in a tuple after ','
get_ipython().set_custom_exc((Exception,), play_sound)  # the sound will be played at any exception thrown

In [18]:
@dataclass
class Flat:
    flatURL: str
    offert_number: str = 'null'
    price: float = 0.0
    price_unit: str = 'zł'
    rent: str = 'null'
    rent_unit: str = 'zł/miesiąc'
    area: str = 'null'
    area_unit: str = 'm^2'
    rooms_number: str = 'null'
    floor: str = 'null'
    max_floor: str = 'null'
    
    def to_list(self):
        return [self.offert_number, self.flatURL, self.price, self.price_unit, 
                self.rent, self.rent_unit, self.area, self.area_unit, 
                self.rooms_number, self.floor, self.max_floor]
    
    def to_dict(self):
        return {'Offert_number': self.offert_number, 'URL': self.flatURL, 'Price': self.price, 'Price_unit': self.price_unit, 
                'Rent_value': self.rent, 'Rent_unit': self.rent_unit, 'Area': self.area, 'Area_unit': self.area_unit, 
                'No._of_rooms': self.rooms_number, 'Floor': self.floor, 'Max_floor': self.max_floor}
    
class Holder(object):
    # https://stackoverflow.com/a/1551223/19716626
    def set(self, value):
        self.value = value
        return value
    def get(self):
        return self.value

In [19]:
def get_flat_info_selenium(driver, count):
    print('Getting info for flat number: ', count)
    regex_digits = re.compile('(\d+,?\.?\d{0,2})(?=\s|$)', re.IGNORECASE)
    regex_unit = re.compile('(?<=\s)([aAaĄąBbCcĆćDdEeĘęFfGgHhIiJjKkLlŁłMmNnŃńOoÓóPpRrSsŚśTtUuWwYyZzŹźŻż]+.*)(?=$)', re.IGNORECASE)
    
    flat = Flat(driver.current_url)
    
    elem = driver.find_elements(By.XPATH,'//main//header/strong[@aria-label="Cena"]')
    #set_trace()
    if elem:
        flat_price_text = elem[0].text
        if flat_price_text and flat_price_text != '':
            price_value = ''
            for m in re.findall(regex_digits, flat_price_text):
                price_value += m
            #flat.price = float(re.match(regex_digits, flat_price).group(0).replace(' ', ''))
            if ',' in price_value:
                price_value = price_value.replace(',', '.')
            if price_value != '':
                #flat.price = float(price_value)
                flat.price = price_value
                flat.price_unit = re.search(regex_unit, flat_price_text).group(0)

    elem = driver.find_elements(By.XPATH, '//div[@aria-label="Czynsz"]')
    if elem:
        elem = driver.find_elements(By.XPATH, '//div[@aria-label="Czynsz"]//div[@class="css-1wi2w6s enb64yk4"]')
        #set_trace()
        if elem:
            flat_rent_text = elem[0].text
            if flat_rent_text and flat_rent_text != '':
                flat_rent_value = ''
                for m in re.findall(regex_digits, flat_rent_text):
                    flat_rent_value += m
                #set_trace()
                if flat_rent_value != '':
                    if ',' in flat_rent_value:
                        flat_rent_value = flat_rent_value.replace(',', '.')
                    flat.rent = flat_rent_value
                    flat.rent_unit=re.search(regex_unit, flat_rent_text).group(0)
                    #set_trace()
        else:
            elem = driver.find_elements(By.XPATH, '//div[@aria-label="Czynsz"]//button[@data-cy="missing-info-button"]')
            if elem:
                flat.rent = 'null'
    
    elem = driver.find_elements(By.XPATH, '//div[@aria-label="Powierzchnia"]')
    if elem:
        elem = driver.find_elements(By.XPATH, '//div[@aria-label="Powierzchnia"]//div[@class="css-1wi2w6s enb64yk4"]')
        #set_trace()
        if elem:
            flat_area_text = elem[0].text
            if flat_area_text and flat_area_text != '':
                flat_area_value = ''
                for m in re.findall(regex_digits, flat_area_text):
                    flat_area_value += m
                if flat_area_value != '':
                    if ',' in flat_area_value:
                        flat_area_value = flat_area_value.replace(',', '.')
                    #flat.area=float(flat_area_value)
                    flat.area=flat_area_value
                    flat.area_unit=re.search(regex_unit, flat_area_text).group(0)
        else:
            elem = driver.find_elements(By.XPATH, '//div[@aria-label="Powierzchnia"]//button[@data-cy="missing-info-button"]')
            if elem:
                flat.area = 'null'
    
    elem = driver.find_elements(By.XPATH, '//div[@aria-label="Liczba pokoi"]')
    if elem:
        elem = driver.find_elements(By.XPATH, '//div[@aria-label="Liczba pokoi"]//div[@class="css-1wi2w6s enb64yk4"]')
        #set_trace()
        if elem:
            flat_rooms_text = elem[0].text
            if flat_rooms_text and flat_rooms_text != '':
                #flat.rooms_number=int(flat_rooms_text)
                flat.rooms_number=flat_rooms_text
        else:
            elem = driver.find_elements(By.XPATH, '//div[@aria-label="Liczba pokoi"]//button[@data-cy="missing-info-button"]')
            if elem:
                flat.rooms_number = 'null'
    
    elem = driver.find_elements(By.XPATH, '//div[@aria-label="Piętro"]')
    if elem:
        elem = driver.find_elements(By.XPATH, '//div[@aria-label="Piętro"]//div[@class="css-1wi2w6s enb64yk4"]')
        #set_trace()
        if elem:
            floors_text = elem[0].text
            if floors_text and floors_text != '':
                floors = []  # floors[0] --> floor number, floors[1] --> max_floors number

                if '/' in floors_text:
                    floors = floors_text.split('/')
                else:
                    #floors = [floors_text, '-100']
                    floors = [floors_text, 'null']

                # to skip all unnecessary whitespaces and great letters
                if 'parter' in floors[0].lower():
                    floors[0] = '0'
                if 'poddasze' in floors[0].lower():
                    #floors[0] = 'poddasze'
                    floors[0] = '99'
                if '> 10' in floors[0].lower():
                    #set_trace()
                    #floors[0] = '> 10'
                    floors[0] = '-10'

                flat.floor=floors[0]
                flat.max_floor = floors[1]
        else:
            elem = driver.find_elements(By.XPATH, '//div[@aria-label="Piętro"]//button[@data-cy="missing-info-button"]')
            if elem:
                flat.floor = 'null'
                flat.max_floor = 'null'
    elem = driver.find_elements(By.XPATH,'//div[contains(text(), "Nr oferty w Otodom:")]')
    #set_trace()
    if elem:
        offert_number_text = elem[0].text
        if offert_number_text and offert_number_text != '':
            flat.offert_number = offert_number_text.lstrip('Nr oferty w Otodom: ')
            
    #set_trace()
    return flat

In [20]:
def get_all_page_offers_selenium(driver, page_number):
    print('Parsing page number: ', page_number)
    #set_trace()
    all_page_offers = []
    
    timeout = 30
    timeout_counter = 0
    while(True):
        try:
            element_present = EC.presence_of_element_located((By.XPATH, '//span[text()="Na stronie"]'))
            WebDriverWait(driver, timeout).until(element_present)
        except TimeoutException:
            print(f'Timed out waiting for page: {page_number} to load.')
            timeout_counter += 1
            if timeout_counter > 4:
                print(f'Failed loading page: {page_number}.')
                break
            else:
                print('Retrying...')
        else:
            all_page_offers_urls = []
            offers_a = driver.find_elements(By.XPATH, '//a[contains(@data-cy, "listing-item-link")]')
            for offer in offers_a:
                all_page_offers_urls.append(offer.get_attribute('href'))

            for count, flat_url in enumerate(all_page_offers_urls):
                #go to each flat page
                timeout = 30
                timeout_counter = 0
                while (True):
                    sleep(randint(5, 10) * 0.1)
                    driver.get(flat_url)
                    try:
                        #print('Waiting for page to load...')
                        element_present = EC.presence_of_element_located((By.XPATH, '//div[contains(@data-cy, "contact-form")]'))
                        WebDriverWait(driver, timeout).until(element_present)
                    except TimeoutException:
                        print(f'Timed out waiting for flat to load: {flat_url}')
                        timeout_counter += 1
                        if timeout_counter > 4:
                            break
                        else:
                            print('Retrying...')    
                    else:
                        all_page_offers.append(get_flat_info_selenium(driver, count+1))
                        #print('Done!')
                        break
            break
            
    return all_page_offers 

In [21]:
def save_offers_to_csv(offers_list, is_rent, where):
    # save all data into a dataframe
    all_offers_dict = {'Offert_no': [flat.offert_number for flat in offers_list], 
                       'URL': [flat.flatURL for flat in offers_list], 
                       'Price': [flat.price for flat in offers_list], 
                       'Price_unit': [flat.price_unit for flat in offers_list], 
                       'Rent': [flat.rent for flat in offers_list], 
                       'Rent_unit': [flat.rent_unit for flat in offers_list], 
                       'Area': [flat.area for flat in offers_list], 
                       'Area_unit': [flat.area_unit for flat in offers_list], 
                       'Rooms_no': [flat.rooms_number for flat in offers_list], 
                       'Floor': [flat.floor for flat in offers_list], 
                       'Max_floor': [flat.max_floor for flat in offers_list]}

    df_offers = pd.DataFrame.from_dict(all_offers_dict, orient='index')
    df_offers = df_offers.transpose()
    df_offers.drop_duplicates(subset=['URL'], keep='first', inplace=True, ignore_index=True)

    #check if the directory to save files exists
    where_plus = ''
    if is_rent:
        where_plus = where + '_flats_rent'
    else:
        where_plus = where + '_flats_sell'
    flats_path = os.path.join('.', where_plus)
    if not os.path.exists(flats_path):
        # create dir
        os.makedirs(flats_path)

    # save dataframe to a csv with a date and hour in a name
    today = date.today()
    csv_file = str(today) + '--otodom_' + where + '_flats.csv'
    if not is_rent:
        csv_file  = 'sell-' + csv_file
    csv_path = os.path.join(flats_path, csv_file)
    print(f'Saving a csv file to {flats_path}...')
    df_offers.to_csv(csv_path, index=None)

In [22]:
def load_main_page(driver, mainURL, is_rent, where):
    if is_rent:
        print(f'\n--- {where.capitalize()}, flats for rent ---')
    else:
        print(f'\n--- Flats for sale ---')
    print('Loading main page...')
    #driver.get(mainURL)
    # wait untill flats hrefs are loaded:
    timeout = 30
    timeout_counter = 0                        
    #set_trace()
    while(True):
        sleep(randint(5, 10) * 0.1)
        driver.get(mainURL)
        try:
            # Look for div where "Wszystkie ogłoszenia" are situated:
            element_present = EC.presence_of_element_located((By.XPATH, '//div[contains(@data-cy, "search.listing.organic")]'))
            WebDriverWait(driver, timeout).until(element_present)
        except TimeoutException:
            print("Timed out waiting for main page to load.")
            timeout_counter += 1
            if timeout_counter > 4:
                break
            else:
                print('Retrying...')
        else:
            # clean from unnecessary buttons, including cookies info:
            akceptuje_button = driver.find_elements(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
            if len(akceptuje_button) > 0:
                akceptuje_button[0].click()
            
            #set_trace()

            offers_total = 0
            wait = WebDriverWait(driver, 30)
            element_present = EC.presence_of_element_located((By.XPATH, '//*[text()="Liczba ogłoszeń: "]'))
            try:
                wait.until(element_present)
            except TimeoutException:
                print('Timed out waiting for no. of offers to load.')
            try:
                elem = driver.find_element(By.XPATH, '//*[text()="Liczba ogłoszeń: "]//following-sibling::span')
            except NoSuchElementException:
                print('Uuups... something is wrong, cannot find offers_total')
            else:
                offers_total_text = elem.text
                offers_total = float(offers_total_text)
            print(f'Total number of offers: {offers_total:.0f}')  

            pages_quantity = ceil(offers_total / 36)  # 36 is the hardcoded limit of offers per page
            print(f'Number of pages to visit: {pages_quantity}.')

            # scroll till the page bottom, to make sure all flat data is loaded
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')

            offers_list = get_all_page_offers_selenium(driver, 1)

            #set_trace()

            if pages_quantity > 1:
                for page_number in range(2, pages_quantity + 1):
                    page_url = pageURL(mainURL, page_number)
                    #sleep(randint(3, 8) * 0.1)
                    #driver.get(page_url)
                    timeout = 30
                    timeout_counter = 0
                    while (True):
                        sleep(randint(5, 10) * 0.1)
                        driver.get(page_url)
                        try:
                            # Look for div where "Wszystkie ogłoszenia" are situated:
                            element_present = EC.presence_of_element_located((By.XPATH, '//div[contains(@data-cy, "search.listing.organic")]'))
                            WebDriverWait(driver, timeout).until(element_present)
                        except TimeoutException:
                            print(f'Timed out waiting for page no. {page_number} to load: {page_url}')
                            timeout_counter += 1
                            if timeout_counter > 4:
                                break
                            else:
                                print('Retrying...')
                        else:
                            # scroll till the page bottom, to make sure all flat data is loaded
                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                            current_offers_list = get_all_page_offers_selenium(driver, page_number)
                            #set_trace()
                            offers_list.extend(current_offers_list)
                            break

            save_offers_to_csv(offers_list, is_rent, where)
            break

In [23]:
def initiate_driver(pageURL, is_rent, where):
    # pageURL - URL where to start crawling
    # is_rent - boolean: 
    #                   True -> flats to rent, 
    #                   False -> flats to sell
    
    
    # set normal loading option
    # (The eager loading options waits until the initial HTML document has been completely loaded and parsed and then discards the rest.)
    # normal loading option waits untill everything is loaded, including javascripts, css etc)
    
    
    options = Options()
    options.page_load_strategy = 'normal'
    options.add_argument('--headless')
    #options.add_argument('--start-maximized')
    options.add_argument("window-size=1920,1080")
    options.add_argument('--disable-gpu')
    #options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
    ua = 'user-agent=' + UserAgent(browsers=['edge', 'chrome']).random
    options.add_argument(ua)

    # Install Webdriver --_ since Selenium v4.6.0 or above no need to use it, enough is to:
    #     from selenium import webdriver
    #     driver = webdriver.Chrome()
    service = Service(ChromeDriverManager().install()) #--> theoretically no longer needed, as told above
    #service = Service()  # --> theoretically would be used instead of ChromeDriverManager().install() above
    #service = Service(ChromeDriverManager(version="114.0.5735.90").install())  # --> temporary solution (?)

    # Open a browser
    #set_trace()
    with webdriver.Chrome(service=service, options=options) as driver:
        load_main_page(driver, pageURL, is_rent, where)
        print('All done!')
        notify

In [24]:
######################################################################################################################
#
# Uncomment the line you want to execute: destination (Ochota or Włochy) and flats for rent, or for sale, or both
#
######################################################################################################################

where = 'ochota'
#where = 'wlochy'

if where == 'ochota':
    initiate_driver(ochota_rentURL, True, where)
    #initiate_driver(ochota_sellURL, False, where)
if where == 'wlochy':
    initiate_driver(wlochy_rentURL, True, where)
    #initiate_driver(wochy_sellURL, False, where)


--- Ochota, flats for rent ---
Loading main page...
Total number of offers: 240
Number of pages to visit: 7.
Parsing page number:  1
Getting info for flat number:  1
Getting info for flat number:  2
Getting info for flat number:  3
Getting info for flat number:  4
Getting info for flat number:  5
Getting info for flat number:  6
Getting info for flat number:  7
Getting info for flat number:  8
Getting info for flat number:  9
Getting info for flat number:  10
Getting info for flat number:  11
Getting info for flat number:  12
Getting info for flat number:  13
Getting info for flat number:  14
Getting info for flat number:  15
Getting info for flat number:  16
Getting info for flat number:  17
Getting info for flat number:  18
Getting info for flat number:  19
Getting info for flat number:  20
Getting info for flat number:  21
Getting info for flat number:  22
Getting info for flat number:  23
Getting info for flat number:  24
Getting info for flat number:  25
Getting info for flat num